[View in Colaboratory](https://colab.research.google.com/github/nailbrainz/tensorflow/blob/master/alexnet_implementation.ipynb)

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_gpus())


print("packs loaded")

['/device:CPU:0', '/device:GPU:0']
packs loaded


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
randidx = np.random.randint(testimg.shape[0], size=150)
print(type(testimg))
print(testimg.shape)
testimg = testimg[randidx, :]
testlabel = testlabel[randidx, :]
print(type(testimg))
print(testimg.shape)
print(type(trainimg))
print(trainimg.shape)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
<class 'numpy.ndarray'>
(10000, 784)
<class 'numpy.ndarray'>
(150, 784)
<class 'numpy.ndarray'>
(55000, 784)


In [0]:
def initial_padding(im):
  ret = np.zeros([227, 227, 3])
  ret[1:226][1:226][:] = im[:][:][:]
  return ret

# Network Building

In [4]:
n_input = trainimg.shape[1]
print(trainimg.shape)
n_output = trainlabel.shape[1]
_input_r = tf.reshape(trainimg, shape=[-1, 28, 28, 1])

x = tf.placeholder(tf.float32, [None, 227, 227, 3])
y = tf.placeholder(tf.float32, [None, 1000])
keepratio = tf.placeholder(tf.float32)

stdev = 0.01
devicetype = "/gpu:0"
with tf.device(devicetype):
  ws = {
      "conv1_w1":tf.Variable(tf.truncated_normal([11, 11, 3, 48], stddev=stdev)),
      "conv1_w2":tf.Variable(tf.truncated_normal([11, 11, 3, 48], stddev=stdev)),
      "conv2_w1":tf.Variable(tf.truncated_normal([5, 5, 48, 128], stddev=stdev)),
      "conv2_w2":tf.Variable(tf.truncated_normal([5, 5, 48, 128], stddev=stdev)),
      "conv3_w1":tf.Variable(tf.truncated_normal([3, 3, 256, 192], stddev=stdev)),
      "conv3_w2":tf.Variable(tf.truncated_normal([3, 3, 256, 192], stddev=stdev)),
      "conv4_w1":tf.Variable(tf.truncated_normal([3, 3, 192, 192], stddev=stdev)),
      "conv4_w2":tf.Variable(tf.truncated_normal([3, 3, 192, 192], stddev=stdev)),
      "conv5_w1":tf.Variable(tf.truncated_normal([3, 3, 192, 128], stddev=stdev)),
      "conv5_w2":tf.Variable(tf.truncated_normal([3, 3, 192, 128], stddev=stdev)),
      "fc1_w":tf.Variable(tf.truncated_normal([(6*6*128*2), 4096], stddev=stdev)),
      "fc2_w":tf.Variable(tf.truncated_normal([4096, 4096], stddev=stdev)),
      "fc3_w":tf.Variable(tf.truncated_normal([4096, 1024], stddev=stdev))
  }
  bs = {
      "conv1_b1":tf.Variable(tf.zeros([48])),
      "conv1_b2":tf.Variable(tf.zeros([48])),
      "conv2_b1":tf.Variable(tf.ones([128])),
      "conv2_b2":tf.Variable(tf.ones([128])),
      "conv3_b1":tf.Variable(tf.zeros([192])),
      "conv3_b2":tf.Variable(tf.zeros([192])),
      "conv4_b1":tf.Variable(tf.ones([192])),
      "conv4_b2":tf.Variable(tf.ones([192])),
      "conv5_b1":tf.Variable(tf.ones([128])),
      "conv5_b2":tf.Variable(tf.ones([128])),
      "fc1_b":tf.Variable(tf.ones([4096])),
      "fc2_b":tf.Variable(tf.ones([4096])),
      "fc3_b":tf.Variable(tf.ones([1024]))
  }
  
  def conv_layer(_layer, _idx, _prevs, _w, _b, _strides, _pad):
    tmp = tf.nn.conv2d(_prevs, _w['conv'+str(_layer)+'_w'+str(_idx)], strides=_strides, padding=_pad)
    tmp = tf.add(tmp, _b['conv'+str(_layer)+'_b'+str(_idx)])
    tmp = tf.nn.relu(tmp)
    return tmp
    
  def resp_normalization(_input, N, C):
    n = 5
    k = 2
    alp = 10e-4
    beta = 0.75
    for g in range(1, 3):
      for i in range(C):
        for x in range(N):
          for y in range(N):
            a1 = 0
            for j in range(max(0, int(i-n/2)), min(C-1, i+int(n/2))):
              a1 += _input[g][x][y][j]**2
            a1 = (k + a1)**beta
            #_input[g][x][y][i].assign(_input[g][x][y][i]/a1)  
  
  def conv_construct(_input, _w, _b, _ratio):
    #input
    #(227 X 227 X 3)
    
    #conv1
    conv1_1 = conv_layer(1, 1, _input, _w, _b, [1, 4, 4, 1], "VALID")
    conv1_2 = conv_layer(1, 2, _input, _w, _b, [1, 4, 4, 1], "VALID")
    # (55 X 55 X 48) X 2
    conv1_1 = tf.nn.max_pool(conv1_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    conv1_2 = tf.nn.max_pool(conv1_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    #conv1_1, conv1_2 = resp_normalization([conv1_1, conv1_2], 55, 48)
    conv1_1 = tf.nn.dropout(conv1_1, _ratio)
    conv1_2 = tf.nn.dropout(conv1_2, _ratio)
    # (27 X 27 X 48) X 2
    
    #conv2
    conv2_1 = conv_layer(2, 1, conv1_1, _w, _b, [1, 1, 1, 1], "SAME")
    conv2_2 = conv_layer(2, 2, conv1_2, _w, _b, [1, 1, 1, 1], "SAME")
    # (27 X 27 X 128) X 2
    conv2_1 = tf.nn.max_pool(conv2_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    conv2_2 = tf.nn.max_pool(conv2_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    #conv2_1, conv2_2 = resp_normalization([conv2_1, conv2_2], N=27, C=128)
    conv2_1 = tf.nn.dropout(conv2_1, _ratio)
    conv2_2 = tf.nn.dropout(conv2_2, _ratio)
    # (13 X 13 X 128) X 2
    
    #conv3
    conv3_input = tf.concat([conv2_1, conv2_2], 3)
    conv3_1 = conv_layer(3, 1, conv3_input, _w, _b, [1, 1, 1, 1], "SAME")
    conv3_2 = conv_layer(3, 2, conv3_input, _w, _b, [1, 1, 1, 1], "SAME")
    # (13 X 13 X 192) X 2
    conv3_1 = tf.nn.dropout(conv3_1, _ratio)
    conv3_2 = tf.nn.dropout(conv3_2, _ratio)
    # (13 X 13 X 192) X 2
    
    #conv4
    conv4_1 = conv_layer(4, 1, conv3_1, _w, _b, [1, 1, 1, 1], "SAME")
    conv4_2 = conv_layer(4, 2, conv3_2, _w, _b, [1, 1, 1, 1], "SAME")
    # (13 X 13 X 192) X 2
    conv4_1 = tf.nn.dropout(conv4_1, _ratio)
    conv4_2 = tf.nn.dropout(conv4_2, _ratio)
    # (13 X 13 X 192) X 2
    
    #conv5
    conv5_1 = conv_layer(5, 1, conv4_1, _w, _b, [1, 1, 1, 1], "SAME")
    conv5_2 = conv_layer(5, 2, conv4_2, _w, _b, [1, 1, 1, 1], "SAME")
    print(conv5_1.get_shape())
    # (13 X 13 X 128) X 2
    conv5_1 = tf.nn.dropout(conv5_1, _ratio)
    conv5_2 = tf.nn.dropout(conv5_2, _ratio)
    conv5_1 = tf.nn.max_pool(conv5_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    conv5_2 = tf.nn.max_pool(conv5_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
    # (11 X 11 X 128 ) X 2
    
    
    print(conv5_2.get_shape())
    fc6_input = tf.concat([conv5_1, conv5_2], 3)
    print(fc6_input.get_shape())
    
    fc6_input = tf.reshape(conv5_1, [-1, int(np.prod(fc6_input.get_shape()[1:]))])
    fc6 = tf.nn.relu(tf.add(tf.matmul(fc6_input, _w['fc1_w']), _b['fc1_b'])) 
    fc7 = tf.nn.relu(tf.add(tf.matmul(fc6, _w['fc2_w']), _b['fc2_b'])) 
    fc8 = tf.nn.relu(tf.add(tf.matmul(fc7, _w['fc3_w']), _b['fc3_b'])) 
    return fc8    

(55000, 784)


In [5]:
keepratio = tf.placeholder(tf.float32)
with tf.device(devicetype):
  pred = conv_construct(x, ws, bs, keepratio)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
  optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
  corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
  accr = tf.reduce_mean(tf.cast(corr, tf.float32))
  init = tf.initialize_all_variables()


(?, 13, 13, 128)
(?, 6, 6, 128)
(?, 6, 6, 256)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [0]:
_config = tf.ConfigProto() #allow_soft_placement=True
sess = tf.Session(config=_config)
sess.run(init)

In [7]:

training_epochs = 15
batch_size = 50
display_step = 1
do_train = 1



for epoch in range(training_epochs):
  total_batch = int(mnist.train.num_examples/batch_size)
  avg_cost = 0
  for i in range(total_batch):
    b_x, b_y = mnist.train.next_batch(batch_size)
    fds = {x:b_x, y:b_y, keepratio:0.7}
    sess.run(optm, feed_dict=fds)
    avg_cost += sess.run(cost, feed_dict=fds)
  avg_cost /= total_batch
  
  if(epoch % display_step == 0):
    print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
    train_acc = sess.run(accr, feed_dict={x:b_x, y:b_y, keepratio:1.})
    print (" Training accuracy: %.3f" % (train_acc))
    test_acc = sess.run(accr, feed_dict={x:testimg, y:testlabel, keepratio:1.})
    print (" Test accuracy: %.3f" % (test_acc))

print ("OPTIMIZATION FINISHED")

ValueError: ignored